In [1]:
%matplotlib inline
%pip install -q mediapy
import mediapy as media
media.set_ffmpeg('/opt/bwhpc/common/vis/ffmpeg/5.1.3/bin/ffmpeg')

Note: you may need to restart the kernel to use updated packages.


In [2]:
from ppo_airhockey_benchmark.test import eval_one, prepare

def start_testing(local_testing, n=15):
    display(local_testing)
    eval_env, model = prepare(local_testing)
    for q in range(50):
        images, cum_reward, reward, info = eval_one(eval_env, model, render_mode='rgb_array')
        print("Run {}".format(q))
        print("Reward until last:{}".format(cum_reward - reward))
        print("Full reward:{}".format(cum_reward))
        media.show_video(images, fps=50, width=500)

/home/kit/stud/uqtlv/.conda/envs/airhockey2/lib/python3.8/site-packages/torch/utils/tensorboard/__init__.py:4: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if not hasattr(tensorboard, "__version__") or LooseVersion(


In [3]:
local_testing = {
    'model_type': 'recurrent',
    'path': 'logs/RnnPPO/7dof_defend/test-recurrent-3',
    'env': '7dof-defend',
    'env_args': {'noise': True, 'horizon': 300, 'last_joint': True}
}

start_testing(local_testing, n=15)

{'model_type': 'recurrent',
 'path': 'logs/RnnPPO/7dof_defend/test-recurrent-3',
 'env': '7dof-defend',
 'env_args': {'noise': True, 'horizon': 300, 'last_joint': True}}

Run 0
Reward until last:[0.33999634]
Full reward:[153.52742]


This browser does not support the video tag.


KeyboardInterrupt: 

In [ ]:
local_testing = {
    'model_type': 'normal',
    'path': 'logs/RnnPPO/7dof_defend/test-ppo-3',
    'env': '7dof-defend',
    'env_args': {'noise': True, 'horizon': 300, 'last_joint': True}
}

start_testing(local_testing, n=15)